In [722]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
import json

from string import punctuation
from scipy.stats import multivariate_normal
from sklearn.feature_extraction.text import TfidfVectorizer

In [612]:
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem.wordnet import WordNetLemmatizer
# nltk.download("punkt")
# nltk.download("averaged_perceptron_tagger")
# nltk.download("wordnet")
# nltk.download("stopwords")
# nltk.download("maxent_ne_chunker")
# nltk.download("words")
# nltk.download("tagsets")

## Data Observe

In [2]:
path = "/Users/brianhung/Documents/myPython/Arc/data/job_postings_training_set.csv"
raw_data = pd.read_csv(path,sep=",")

In [4]:
raw_data.columns

Index(['job_id', 'title', 'location', 'department', 'salary_range',
       'company_profile', 'description', 'requirements', 'benefits',
       'telecommuting', 'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')

In [5]:
raw_data.describe(include = "all")

,job_id,title,location,department,salary_range,company_profile,description,requirements,benefits,telecommuting,has_company_logo,has_questions,employment_type,required_experience,required_education,industry,function,fraudulent
count,17828.000000,17828,17483,6314,2862,14538,17827,15140,10644,17828.000000,17828.000000,17828.000000,14370,10804,9755,12944,11395,17828.000000
unique,NaN,11188,3097,1337,874,1707,14677,11881,6131,NaN,NaN,NaN,5,7,13,131,37,NaN
top,NaN,English Teacher Abroad,"GB, LND, London",Sales,0-0,We help teachers get safe &amp; secure jobs ab...,"Play with kids, get paid for it Love travel? J...",University degree required. TEFL / TESOL / CEL...,See job description,NaN,NaN,NaN,Full-time,Mid-Senior level,Bachelor's Degree,Information Technology and Services,Information Technology,NaN
freq,NaN,311,717,547,141,726,380,410,726,NaN,NaN,NaN,11588,3796,5131,1729,1743,NaN
mean,8914.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.042966,0.795434,0.491979,NaN,NaN,NaN,NaN,NaN,0.048351
std,5146.644635,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.202786,0.403395,0.499950,NaN,NaN,NaN,NaN,NaN,0.214513
min,1.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000
25%,4457.750000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000
50%,8914.500000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,0.000000,NaN,NaN,NaN,NaN,NaN,0.000000
75%,13371.250000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000000,1.000000,1.000000,NaN,NaN,NaN,NaN,NaN,0.000000


In [132]:
raw_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17828 entries, 0 to 17827
Data columns (total 18 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   job_id               17828 non-null  int64 
 1   title                17828 non-null  object
 2   location             17483 non-null  object
 3   department           6314 non-null   object
 4   salary_range         2862 non-null   object
 5   company_profile      14538 non-null  object
 6   description          17827 non-null  object
 7   requirements         15140 non-null  object
 8   benefits             10644 non-null  object
 9   telecommuting        17828 non-null  int64 
 10  has_company_logo     17828 non-null  int64 
 11  has_questions        17828 non-null  int64 
 12  employment_type      14370 non-null  object
 13  required_experience  10804 non-null  object
 14  required_education   9755 non-null   object
 15  industry             12944 non-null  object
 16  func

In [7]:
raw_data.corr()

,job_id,telecommuting,has_company_logo,has_questions,fraudulent
job_id,1.000000,-0.003990,-0.014058,-0.086643,0.079736
telecommuting,-0.003990,1.000000,-0.020094,0.019998,0.034770
has_company_logo,-0.014058,-0.020094,1.000000,0.234263,-0.261673
has_questions,-0.086643,0.019998,0.234263,1.000000,-0.091578
fraudulent,0.079736,0.034770,-0.261673,-0.091578,1.000000


## EDA

- **'job_id': --int**  
不處理
- **'title': --text**  
The title of the job ad entry  
toDo:分離title詞向量做cosine similarity後分群，群數為10？要測試
- **'location': --text**  
Geographical location of the job ad  
分離出「country」、「state」、「location」、...
- **'department': --text**  
Corporate department (e.g. sales)  
toDo
- **'salary_range': --text**  
Indicative salary range (e.g. $50,000-$60,000)  
分離薪資上下限 e.g. 90000-150000 -> min_salary:90000,max_salary:150000
- **'company_profile': --text**  
A brief company description  
方法toDo
- **'description': --text**  
The details description of the job ad  
方法toDo
- **'requirements': --text**  
Enlisted requirements for the job opening  
方法toDo
- **'benefits': --text**  
Enlisted offered benefits by the employer  
方法toDo
- **'telecommuting': --bool**  
True for telecommuting positions  
[0,1]
- **'has_company_logo': --bool**  
True if company logo is present  
[0,1]
- **'has_questions': --bool**  
True if screening questions are present  
[0,1]
- **'employment_type': --text**  
Full-type, Part-time, Contract, etc  
['Contract', 'Full-time', 'Other', 'Part-time', 'Temporary',None]
- **'required_experience': --text**  
Executive, Entry level, Intern, etc  
['Associate', 'Director', 'Entry level', 'Executive', 'Internship', 'Mid-Senior level', 'Not Applicable']
- **'required_education': --text**  
Doctorate, Master’s Degree, Bachelor, etc  
['Associate Degree', 'Bachelor's Degree', 'Certification', 'Doctorate', 'High School or equivalent', 'Master's Degree', 'Professional', 'Some College Coursework Completed', 'Some High School Coursework', 'Unspecified', 'Vocational', 'Vocational - Degree', 'Vocational - HS Diploma']
- **'industry': --text**  
Automotive, IT, Health care, Real estate, etc  
['Accounting', 'Airlines/Aviation', 'Alternative Dispute Resolution', 'Animation', 'Apparel & Fashion', 'Architecture & Planning', 'Automotive', 'Aviation & Aerospace', 'Banking', 'Biotechnology',  
...  
'Translation and Localization', 'Transportation/Trucking/Railroad', 'Utilities', 'Venture Capital & Private Equity', 'Veterinary', 'Warehousing', 'Wholesale', 'Wine and Spirits', 'Wireless', 'Writing and Editing']
- **'function': --text**  
Consulting, Engineering, Research, Sales etc  
功能性?市場 -> 跟column['department']類似?
- **'fraudulent': --int**  
[0,1]

### - location

In [241]:
"""
後續可以用各國location data 過濾資料
"""
location_data = raw_data[['job_id','location']].copy()

# Fill NaN and non-string -> ''
location_data['location'] = [i if type(i)==str else '' for i in location_data['location']]

# Split the location data into list() -> location['loc_list']
loc_list = []
for i in range(len(location_data)):
    tmp = location_data['location'][i]
    if tmp == '':
        loc_list.append('')
        continue

    tmp_loc = tmp.split(sep = ',', maxsplit = -1)

    # drop empty string and trim(strip)
    tmp_loc = [i.strip() for i in tmp_loc if i.strip() != '']
    loc_list.append(tmp_loc)

location_data['loc_list'] = loc_list

# Use set() to save all the locations in loc_set
loc_set = set()
for i in loc_list:
    loc_set.update(i)
    
# location_data explode all locations to columns
location_data = pd.concat([location_data,pd.DataFrame(columns=list(loc_set))])
location_data = location_data.fillna(0)

# Update the location data from row
for i in range(len(location_data)):
    one_loc_list = location_data.loc[i,'loc_list']
    for loc in one_loc_list:
        location_data.loc[i,loc] = 1

### - department

In [276]:
"""
1. 後續可用job_title,job_description,industry,function...等欄位填補NaN
2. 若有對應的"職稱library",可簡化職稱資料
   e.g. kaggle job title library?
"""

department_data = raw_data[['job_id','department']].copy()

# Fill NaN and non-string -> ''
department_data['department'] = [i if type(i)==str else '' for i in department_data['department']]

# Use set() to save all the departments in dep_set
dep_list = list(department_data['department'])
dep_set = set()
dep_set.update(dep_list)

# Clean type of int data -> ''
for i in range(len(department_data)):
    try:
        int(department_data.loc[i,'department'])
        department_data.loc[i,'department'] = ''
    except:
        continue


### - salary_range

In [420]:
def check_salary(salary_list):
    """
    Check if salary_list's elements is all Numbers
    """
    rel = True
    for i in salary_list:
        if not i.isnumeric():
            rel = False
    return rel

In [349]:
"""
後續若有薪資水平評估，可依salary range做薪水高低的分類
"""
salary_data = raw_data[['job_id','salary_range']].copy()

# Fill NaN and non-string -> '0-0'
salary_data['salary_range'] = [i if type(i)==str else '0-0' for i in salary_data['salary_range']]

# ToDo: description
min_salary = []
max_salary = []
avg_salary = []
for i in range(len(salary_data)):
    tmp_rel = ['0','0','0'] #[tmp_min, tmp_max, tmp_avg]
    tmp = salary_data['salary_range'][i]
    tmp_salary = tmp.split(sep = '-', maxsplit = -1)
    
    if check_salary(tmp_salary):
        if len(tmp_salary) == 1:
            tmp_rel = [int(tmp_salary[0])]*3
        if len(tmp_salary) == 2:
            tmp_rel = tmp_salary + [str(int((int(tmp_min) + int(tmp_max))/2))]
    
    min_salary.append(tmp_rel[0])
    max_salary.append(tmp_rel[1])
    avg_salary.append(tmp_rel[2])

salary_data['min_salary'] = min_salary
salary_data['max_salary'] = max_salary
salary_data['avg_salary'] = avg_salary

### text mining with nltk & tfidf

In [720]:
# nltk NER table
# nltk.help.upenn_tagset()

### - clean data & cut word function set

In [721]:
# functions for cleaning data
def remove_urls(s):
    s = re.sub('[^\s]*.com[^\s]*', "", s)
    s = re.sub('[^\s]*www.[^\s]*', "", s)
    s = re.sub('[^\s]*.co.uk[^\s]*', "", s)
    return s

def remove_star_words(s):
    return re.sub('[^\s]*[\*]+[^\s]*', "", s)

def remove_nums(s):
    return re.sub('[^\s]*[0-9]+[^\s]*', "", s)

def remove_punctuation(s):
    """
    1. Cutumize replace the dash '-' -> ' '
    2. Replace other symbols -> ''
    """
    s = s.replace('-',' ')
    s = s.replace('–','')
    global punctuation
    for p in punctuation:
        s = s.replace(p, '')

    return s

def convert_lower_case(s):
    return s.lower()

def remove_stopwords(s):
    global en_stopwords
    s = word_tokenize(s)
    s = " ".join([w for w in s if w not in en_stopwords])
    return s

def filter_text(s):
    """
    Filter text with ADJ,VERB,NOUN,ADV
    """
    #tokenize words
    tokenized_corpus = nltk.word_tokenize(s)
    
    # Lemmatizer(字型還原)
    lemmatized_tokens = [lmtzr.lemmatize(token) for token in tokenized_corpus]
    
    # 詞性標記
    pos = nltk.pos_tag(lemmatized_tokens)
    
    wordnet_pos = []
    for p in pos:
        word, tag = p
        if tag.startswith('J'):
            wordnet_pos.append(nltk.corpus.wordnet.ADJ)
        elif tag.startswith('V'):
            wordnet_pos.append(nltk.corpus.wordnet.VERB)
        elif tag.startswith('N'):
            wordnet_pos.append(nltk.corpus.wordnet.NOUN)
        elif tag.startswith('R'):
            wordnet_pos.append(nltk.corpus.wordnet.ADV)
        else:
            wordnet_pos.append('')
    
    # Lemmatizer(字型還原)
    lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
    tokens_le = [lemmatizer.lemmatize(pos[i][0], pos=wordnet_pos[i]) 
                 for i in range(len(pos)) if wordnet_pos[i] != '']
    
    return tokens_le

In [733]:
def main_clean_data(df):
    """
    Mainly function to clean data and EDA with nltk.
    input: Pandas DataFrame with one column
    output: Pandas DataFrame with one column
    """
    clean_data = df
    en_stopwords = stopwords.words('english') + ['’','amp']

    fun_clean_data_list = [remove_urls,
                           remove_star_words,
                           remove_nums,
                           remove_punctuation,
                           convert_lower_case,
                           remove_stopwords,
                           filter_text
                          ]
    
    for func in fun_clean_data_list:
        clean_data = clean_data.map(func)
    return clean_data

### - title

In [ ]:
"""
1. Use NB to predict the loss ['title'] from other column 
like ['company_profile','description','requirements',...]
"""
title_data = raw_data[['job_id','title']]

# Fill NaN
title_data = title_data.fillna('')

# Clean and cut word
title_data['clean_text_tag'] = main_clean_data(title_data['title'])

### - company_profile

In [734]:
company_profile_data = raw_data[['job_id','company_profile']]

# Fill NaN
company_profile_data = company_profile_data.fillna('')

# Clean and cut word
company_profile_data['clean_text_tag'] = main_clean_data(company_profile_data['company_profile'])

### - description

In [738]:
description_data = raw_data[['job_id','description']]

# Fill NaN
description_data = description_data.fillna('')

# Clean and cut word
description_data['clean_text_tag'] = main_clean_data(description_data['description'])

### - toDo: description 的 TFIDF

In [ ]:
# TF
# 10 most common words in the job description
# import the necessary functions from the nltk library

lmtzr = WordNetLemmatizer()

# prepare corpus from the descriptions that dont have stopwords
# tmp_description = description_data['description'].tolist()
# tmp_description = [i if type(i)==str else '' for i in tmp_description ]
corpus = " ".join(description_data['clean_full_description_no_stop'].tolist())

#tokenize words
tokenized_corpus = nltk.word_tokenize(corpus)

# lemmatize these tokens
lemmatized_tokens = [lmtzr.lemmatize(token) for token in tokenized_corpus]



# grammer - <NN>* or <JJ>*<NN>
# grammer = r"""
#           NP: {<JJ>*<NN>}   # chunk determiner/possessive, adjectives and noun
#               {<NNP>+}                # chunk sequences of proper nouns
#           """
# cp = nltk.RegexpParser(grammar)
# sentence = [("Rapunzel", "NNP"), ("let", "VBD"), ("down", "RP"), [1]
#                  ("her", "PP$"), ("long", "JJ"), ("golden", "JJ"), ("hair", "NN")]

# word frequencies for the lemmatized tokens
fd = nltk.FreqDist(lemmatized_tokens)

# get the top words
top_words = []
for key, value in fd.items():
    top_words.append((key, value))

# sort the list by the top frequencies
top_words = sorted(top_words, key = lambda x:x[1], reverse = True)

# keep top 10 words only
top_10_words = top_words[:10]

top_10_words

### - requirements

In [741]:
requirements_data = raw_data[['job_id','requirements']]

# Fill NaN
requirements_data = requirements_data.fillna('')

# Clean and cut word
requirements_data['clean_text_tag'] = main_clean_data(requirements_data['requirements'])

### - benefits

In [742]:
benefits_data = raw_data[['job_id','benefits']]
# Fill NaN
benefits_data = benefits_data.fillna('')

# Clean and cut word
benefits_data['clean_text_tag'] = main_clean_data(benefits_data['benefits'])

### - telecommuting

In [353]:
telecommuting_data = raw_data[['job_id','telecommuting']].copy()

### - has_company_logo

In [356]:
has_company_logo_data = raw_data[['job_id','has_company_logo']].copy()

### - has_questions

In [360]:
has_questions_data = raw_data[['job_id','has_questions']].copy()

### - employment_type

In [371]:
employment_type_data = raw_data[['job_id','employment_type']].copy()

# Fill NaN and non-string -> 'Unknown'
employment_type_data['employment_type'] = [i if type(i)==str else 'Unknown' 
                                           for i in employment_type_data['employment_type']]

### - required_experience

In [377]:
required_experience_data = raw_data[['job_id','required_experience']].copy()

# Fill NaN and non-string -> 'Unknown'
required_experience_data['required_experience'] = [i if type(i)==str else 'Unknown' 
                                                   for i in required_experience_data['required_experience']]

### - required_education

In [388]:
"""
1. 這裡的null值，不一定是資料中的'Unspecified'，所以歸類為'Unknown'
2. 可能可以從job desciption中回填,但需要education的詞庫
"""
required_education_data = raw_data[['job_id','required_education']].copy()

# Fill NaN and non-string -> 'Unknown'
required_education_data['required_education'] = [i if type(i)==str else 'Unknown' for i in required_education_data['required_education']]

### - industry

In [418]:
"""
1. 這裡的null值，歸類為'Unknown'
2. 可能可以從job desciption中回填,但需要industry的詞庫
"""
industry_data = raw_data[['job_id','industry']].copy()

# Fill NaN and non-string -> 'Unknown'
industry_data['industry'] = [i if type(i)==str else 'Unknown' for i in industry_data['industry']]

### - function

In [412]:
"""
1. 這裡的null值，歸類為'Unknown'
2. 可能可以從job desciption中回填,但需要industry的詞庫
"""
function_data = raw_data[['job_id','function']].copy()

# Fill NaN and non-string -> 'Unknown'
# function_data['function'] = [i if type(i)==str else 'Unknown' for i in function_data['function']]